In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('train.csv')

In [3]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder 
from category_encoders import HashingEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [4]:
num_feat = ['Age','Annual_Premium']
cat_feat = ['Gender', 'Vehicle_Damage', 'Vehicle_Age','Driving_License', 'Previously_Insured','Policy_Sales_Channel']

In [5]:
 def encode():
  
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])

    numeric_transformer = Pipeline(
        steps = [
            ("scaler", StandardScaler())]
    )
    
    
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", numeric_transformer, num_feat),
            ("onehot", categorical_transformer, cat_feat)
        ]
    )

    clf = Pipeline(
        steps=[("preprocessor", preprocessor)]
    )

    return clf

In [6]:
def returnXY(): 
    
    clf = encode()

    clf.fit(train_df)

    train_x = clf.transform(train_df[num_feat + cat_feat])
    test_x = clf.transform(test_df[num_feat + cat_feat])
    
    train_y = train_df['Response']
    test_y = test_df['Response']

    return train_x, train_y , test_x , test_y 

In [ ]:
#cat_cols = clf.named_steps['preprocessor'].transformers_[1][1] \
#    .named_steps['onehot'].get_feature_names_out(cat_feat)
    
#X = pd.DataFrame(ds.toarray(), columns=num_feat + cat_cols.tolist())

In [7]:
train_x , train_y ,test_x, test_y = returnXY()